#### Python code for Data Extraction (Wiser solutions technical assignment)
#### By Ameya Ghatpande on Jan-20-2018

In [535]:
# <aghatpande> on 20-Jan-2018
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [536]:
# <aghatpande> on 20-Jan-2018
# Import all the libraries
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests

In [537]:
# <aghatpande> on 20-Jan-2018
# Get the html page and convert it to the Beautiful Soup object
nationalFootballLeaguePage = requests.get('https://en.wikipedia.org/wiki/National_Football_League')
nationalFootballLeaguePage
soup = BeautifulSoup(nationalFootballLeaguePage.content, 'html.parser')
print type(soup)

<Response [200]>

<class 'bs4.BeautifulSoup'>


In [538]:
#print(soup)

#### Q.1 ] Get all the images in the HTML page

In [539]:
# <aghatpande> on 20-Jan-2018
# Find all the images in the html document
# Images are identified by the tag <a> and class <image>
pictures = soup.find_all('a', class_="image")

In [540]:
#print(pictures)

In [541]:
# <aghatpande> on 20-Jan-2018
# Images are stored as relative path. But we need to show to complete path
# Adding the prefix to each image
prefix = "https://en.wikipedia.org/wiki/National_Football_League#/media/"

In [542]:
# <aghatpande> on 20-Jan-2018
# User defined function to extract the href part of image, prepocess it and store it in a List
def getImageLinks(images):
    for image in images:
        href = image['href']
        hrefString = str(href).lstrip("/wiki")
        hrefString = prefix + hrefString
        imageLinks.append(hrefString)
        print(hrefString)

In [543]:
# <aghatpande> on 20-Jan-2018
# Print all the Image links in the webpage
getImageLinks(pictures)

https://en.wikipedia.org/wiki/National_Football_League#/media/File:National_Football_League_logo.svg
https://en.wikipedia.org/wiki/National_Football_League#/media/File:1920_akron_pros_posing.jpg
https://en.wikipedia.org/wiki/National_Football_League#/media/File:Roger_Goodell_(cropped).jpg
https://en.wikipedia.org/wiki/National_Football_League#/media/File:US_National_Football_League_Teams_Location-en.svg
https://en.wikipedia.org/wiki/National_Football_League#/media/File:NFL_playoff_tree.PNG
https://en.wikipedia.org/wiki/National_Football_League#/media/File:Myles_Garrett_2017_preseason_(3)_(cropped).jpg
https://en.wikipedia.org/wiki/National_Football_League#/media/File:Map_of_USA,_NFL.svg


#### Web scrap the HTML Table into a Pandas Dataframe

In [544]:
# <aghatpande> on 20-Jan-2018
# Lets create a Pandas dataframe out of the HTML table 
gamesTable = soup.find_all('table', class_="navbox plainrowheaders wikitable")
# Make sure we get only 1 HTML table
# len(gamesTable)
# type(gamesTable)
# Extract the bs4.element.Tag from bs4.element.ResultSet
gamesTable = gamesTable[0]
# type(gamesTable)

# Lets read the Header of the table. It is the first row
tmp = gamesTable.find_all('tr')
first = tmp[0]   

# All remaining rows have the Table values 
allRows = tmp[1:-1]

# Get the header (column) names
headers = [header.get_text() for header in first.find_all('th')]
# headers

# Get the rows which has column values corresponding to the header olumns
results = [[data.get_text() for data in row.find_all('td')] for row in allRows]
# results

# Our table has rowspans. Lets handle this and fill (repeat) the value of rowspan across rows.
rowspan = []
for no, tr in enumerate(allRows, 0):
    tmp = []
    for th_no, data in enumerate(tr.find_all('th'), 0):
        #print  data.has_key("rowspan")
        if data.has_key("rowspan"):
            rowspan.append((no, th_no, int(data["rowspan"]), data.get_text()))
            
# rowspan

# Fill the repetitive values for rowspans
if rowspan:
    for i in rowspan:
        # tr value of rowspan in present in 1th place in results
        for j in xrange(0, i[2]):
            # Add value in next tr.
            results[i[0]+j].insert(i[1], i[3])

# Create a Pandas DataFrame
footballLeagueDF = pd.DataFrame(data=results, columns=headers)

# Drop the Null columns
footballLeagueDF.dropna()

,Division[55],Club[55],City,Stadium[56],Capacity,Coordinates,First season[57],Head coach
1,East,Buffalo Bills,"Orchard Park, NY",New Era Field,"71,608",42°46′26″N 78°47′13″W﻿ / ﻿42.774°N 78.787°W﻿ /...,"000000001960-01-01-00001960 (AFL), 00000000197...",Sean McDermott
2,East,Miami Dolphins,"Miami Gardens, FL",Hard Rock Stadium,"65,326",25°57′29″N 80°14′20″W﻿ / ﻿25.958°N 80.239°W﻿ /...,"000000001966-01-01-00001966 (AFL), 00000000197...",Adam Gase
3,East,New England Patriots,"Foxborough, MA",Gillette Stadium,"66,829",42°05′28″N 71°15′50″W﻿ / ﻿42.091°N 71.264°W﻿ /...,"000000001960-01-01-00001960 (AFL), 1970 (NFL)",Bill Belichick
4,East,New York Jets,"East Rutherford, NJ",MetLife Stadium[B],"82,500",40°48′50″N 74°04′26″W﻿ / ﻿40.814°N 74.074°W﻿ /...,"000000001960-01-01-00001960 (AFL), 00000000197...",Todd Bowles
5,North,Baltimore Ravens,"Baltimore, MD",M&T Bank Stadium,"71,008",39°16′41″N 76°37′23″W﻿ / ﻿39.278°N 76.623°W﻿ /...,000000001996-01-01-00001996[C],John Harbaugh
6,North,Cincinnati Bengals,"Cincinnati, OH",Paul Brown Stadium,"65,515",39°05′42″N 84°30′58″W﻿ / ﻿39.095°N 84.516°W﻿ /...,"000000001968-01-01-00001968 (AFL), 00000000197...",Marvin Lewis
7,North,Cleveland Browns,"Cleveland, OH",FirstEnergy Stadium,"67,431",41°30′22″N 81°41′56″W﻿ / ﻿41.506°N 81.699°W﻿ /...,"000000001946-01-01-00001946 (AAFC), 0000000019...",Hue Jackson
8,North,Pittsburgh Steelers,"Pittsburgh, PA",Heinz Field,"68,400",40°26′49″N 80°00′58″W﻿ / ﻿40.447°N 80.016°W﻿ /...,000000001933-01-01-00001933,Mike Tomlin
9,South,Houston Texans,"Houston, TX",NRG Stadium,"72,220",29°41′06″N 95°24′40″W﻿ / ﻿29.685°N 95.411°W﻿ /...,000000002002-01-01-00002002,Bill O'Brien
10,South,Indianapolis Colts*,"Indianapolis, IN",Lucas Oil Stadium,"67,000",39°45′36″N 86°09′50″W﻿ / ﻿39.760°N 86.164°W﻿ /...,000000001953-01-01-00001953,Vacant


In [545]:
# <aghatpande> on 20-Jan-2018
# Rename the columns to friendly name
footballLeagueDF.rename(columns={'Division[55]': 'Division', 'Club[55]': 'Club', 'First season[57]':'First season', 'Stadium[56]':'Stadium'}, inplace=True)

#### Q.2 ] Find the teams that are based out of South Division

In [546]:
# <aghatpande> on 20-Jan-2018
# Find the teams based out of South Division
southDivisionTeams = footballLeagueDF[footballLeagueDF.Division == "South"]

In [547]:
southDivisionTeams[["Division", "Club"]]

,Division,Club
9,South,Houston Texans
10,South,Indianapolis Colts*
11,South,Jacksonville Jaguars
12,South,Tennessee Titans*
26,South,Atlanta Falcons
27,South,Carolina Panthers
28,South,New Orleans Saints
29,South,Tampa Bay Buccaneers


#### Q.3] Find the stadium with capacity between 50k and 80k

In [548]:
# <aghatpande> on 20-Jan-2018
# Lets find the Stadiums with Capacity between 50 and 80k
mediumCapacityStadiumsDF = footballLeagueDF[((footballLeagueDF.Capacity >= "50000") & (footballLeagueDF.Capacity <= "80000"))]

In [549]:
mediumCapacityStadiumsDF[["Stadium", "Capacity"]]

,Stadium,Capacity
1,New Era Field,"71,608"
2,Hard Rock Stadium,"65,326"
3,Gillette Stadium,"66,829"
5,M&T Bank Stadium,"71,008"
6,Paul Brown Stadium,"65,515"
7,FirstEnergy Stadium,"67,431"
8,Heinz Field,"68,400"
9,NRG Stadium,"72,220"
10,Lucas Oil Stadium,"67,000"
11,EverBank Field[D],"67,246"


#### Q.4] Find the teams which are based out in following coordinates (Above 39N and East of 84W)

In [550]:
# <aghatpande> on 20-Jan-2018
# Split the coordinates from degree-minutes-seconds format to degree-decimal format
def getNorthCoordinatesInDecimal(cood):
    northCoordinatesInDecimal = None
    if cood == None:
        return northCoordinatesInDecimal
    coordinatesInDecimal = cood.split('/')[2]
    coordinatesInDecimal = coordinatesInDecimal.split('(')[0]
    northCoordinatesInDecimal= coordinatesInDecimal.split(';')[0]
    return northCoordinatesInDecimal

def getWestCoordinatesInDecimal(cood):
    westCoordinatesInDecimal = None
    if cood == None:
        return westCoordinatesInDecimal
    coordinatesInDecimal = cood.split('/')[2]
    coordinatesInDecimal = coordinatesInDecimal.split('(')[0]
    westCoordinatesInDecimal= coordinatesInDecimal.split(';')[1]
    return westCoordinatesInDecimal
  
footballLeagueDF['NorthCoordinate'] = footballLeagueDF["Coordinates"].apply(lambda x: getNorthCoordinatesInDecimal(x))
footballLeagueDF['WestCoordinate'] = footballLeagueDF["Coordinates"].apply(lambda x: getWestCoordinatesInDecimal(x))
footballLeagueDF.dropna(inplace=True)

In [551]:
# <aghatpande> on 20-Jan-2018
# Convert the North coordinate and West coordinate columns to int, so that we can compare values
def convertNorthCordinateToInt(x):
    x = float(x)
    return int(x)

def convertWestCoordinateInt(x):
    m = re.search(r'\d+', x)
    numeric = m.group() # retrieve numeric string
    return int(numeric)

footballLeagueDF['NorthCoordinate'] = footballLeagueDF["NorthCoordinate"].apply(lambda x: convertToInt(x))
footballLeagueDF['WestCoordinate'] = footballLeagueDF["WestCoordinate"].apply(lambda x: convertWestCoordinateInt(x))

In [552]:
# <aghatpande> on 20-Jan-2018
# Find teams North of 38N degrees and East of 84W
targetClubs = footballLeagueDF[((footballLeagueDF.NorthCoordinate > 39) & (footballLeagueDF.WestCoordinate <84))]

In [553]:
targetClubs[["Club", "NorthCoordinate", "WestCoordinate"]]

,Club,NorthCoordinate,WestCoordinate
1,Buffalo Bills,42,78
3,New England Patriots,42,71
4,New York Jets,40,74
7,Cleveland Browns,41,81
8,Pittsburgh Steelers,40,80
19,New York Giants,40,74
23,Detroit Lions*,42,83


### Thank you.
#### Ameya Ghatpande on Jan-21-2018